In [1]:
from selenium import webdriver
import time
import pandas as pd
from riotwatcher import LolWatcher, ApiError
from random import randrange
import time

In [2]:
browser = webdriver.Chrome(executable_path="./chromedriver.exe",keep_alive=True)
browser.get("https://web.whatsapp.com/")
time.sleep(60 * 5)

# Rename id for group, keyboard and enter

In [3]:
def send_message(text):
    browser.find_element_by_id("novid19").click()
    time.sleep(randrange(5))
    browser.find_element_by_id("keyboard").send_keys(text)
    time.sleep(randrange(3))
    browser.find_element_by_id("enter").click()

send_message('....')

## riot_watcher

In [4]:
# golbal variables
api_key = 'API_KEY_HERE'
watcher = LolWatcher(api_key)
my_region = 'euw1'
me = watcher.summoner.by_name(my_region, 'bobo+carry')

### get latest timestamp game

In [ ]:
def getMatchInfo(me):
    my_matches = watcher.match.matchlist_by_account(my_region, me['accountId'])
    return my_matches

def getLatestTimeStamp(my_matches):
    last_match = my_matches['matches'][0]
    return last_match['timestamp']

def getId(pIdent, name):
    for i in pIdent:
        if i['player']['summonerName'] == name:
            return i['participantId']
    
    return 'not found'

def getWin(match_detail, name):
    for player in match_detail['participants']:
        if player['participantId'] == getId(match_detail['participantIdentities'], 'bobo carry'):
            return player['stats']['win']
    return 'not found'

track_start = getLatestTimeStamp(getMatchInfo(me))
# track_start = 1607035178741

loseStreak = 0

while(True):
    played_match = watcher.match.matchlist_by_account(my_region, me['accountId'], begin_time=track_start)
    if len(played_match['matches']) > 1:
        total_duration = 0
        for match in list(reversed(played_match['matches']))[1:]:
            match_detail = watcher.match.by_id(my_region, match['gameId'])
            total_duration += match_detail['gameDuration']
            win = getWin(match_detail, 'bobo carry')
            if not win:
                loseStreak += 1
            else:
                loseStreak = 0
        total_duration = time.strftime('%H:%M:%S', time.gmtime(total_duration))
        text = f'NSA (Norman Security Agent): Borkan hat vor kurzem {played_match["totalGames"] - 1} Spiel(e) gespielt. Insgesamt für {total_duration} gespielt.'
        if loseStreak > 2:
            text += f' \U0001F4A9 Losing streak: {loseStreak} Spiele \U0001F4A9'
        send_message(text)
        track_start = played_match['matches'][0]['timestamp']
    time.sleep(60 * 60)